In [1]:
!cp -r ../input/recsys-repo/RecSys_Course_AT_PoliMi-master/* ./

In [2]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [3]:
#load data_train, matrix of users interaction
URM_path = "../input/recommender-system-2021-challenge-polimi/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                header=0)
URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]

In [ ]:
URM_all_dataframe.head()

In [4]:
userID_unique = URM_all_dataframe["UserID"].unique()
itemID_unique = URM_all_dataframe["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

Number of items	 18059, Number of users	 13650
Max ID items	 18058, Max Id users	 13649

Average interactions per user 387.23
Average interactions per item 292.69

Sparsity 97.86 %


In [5]:
URM_all = sps.coo_matrix((URM_all_dataframe["Interaction"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)
URM_all

<13650x18059 sparse matrix of type '<class 'numpy.float64'>'
	with 5285664 stored elements in Compressed Sparse Row format>

In [7]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

# split data into train and validation data 80/20
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [8]:
from Evaluation.Evaluator import EvaluatorHoldout

#create an evaluator object to evaluate validation set
#we will use it for hyperparameter tuning
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13641 ( 0.1%) Users that have less than 1 test interactions


In [9]:
import numpy as np
import scipy.sparse as sps
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
import time, sys
from tqdm import tqdm
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# os.environ["PYTHONWARNINGS"] = ('ignore::exceptions.ConvergenceWarning:sklearn.linear_model')
# os.environ["PYTHONWARNINGS"] = ('ignore:Objective did not converge:ConvergenceWarning:')

class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available
    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.
        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    @ignore_warnings(category=ConvergenceWarning)
    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100,**earlystopping_kwargs):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)

        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        # Use array as it reduces memory requirements compared to lists
        dataBlock = 10000000

        rows = np.zeros(dataBlock, dtype=np.int32)
        cols = np.zeros(dataBlock, dtype=np.int32)
        values = np.zeros(dataBlock, dtype=np.float32)

        numCells = 0

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values

            # Select topK values
            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index

            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            local_topK = min(len(nonzero_model_coef_value)-1, self.topK)

            relevant_items_partition = (-nonzero_model_coef_value).argpartition(local_topK)[0:local_topK]
            relevant_items_partition_sorting = np.argsort(-nonzero_model_coef_value[relevant_items_partition])
            ranking = relevant_items_partition[relevant_items_partition_sorting]

            for index in range(len(ranking)):

                if numCells == len(rows):
                    rows = np.concatenate((rows, np.zeros(dataBlock, dtype=np.int32)))
                    cols = np.concatenate((cols, np.zeros(dataBlock, dtype=np.int32)))
                    values = np.concatenate((values, np.zeros(dataBlock, dtype=np.float32)))


                rows[numCells] = nonzero_model_coef_index[ranking[index]]
                cols[numCells] = currentItem
                values[numCells] = nonzero_model_coef_value[ranking[index]]

                numCells += 1

            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        # generate the sparse weight matrix
        self.W_sparse = sps.csr_matrix((values[:numCells], (rows[:numCells], cols[:numCells])),
                                       shape=(n_items, n_items), dtype=np.float32)

In [ ]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [ ]:
#let's fit the model with the hyperparamethers obtained from the previous search and evaluate them on validation set

recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_train)
recommender_SLIMElasticNet.fit(epochs = 500, l1_ratio = 0.0023170159712850467, alpha = 0.09078974149197175, 
                positive_only = True, topK = 363)
#evaluator_valid.evaluateRecommender(recommender)

In [ ]:
recommender_SLIMElasticNet.save_model(output_folder_path, file_name = recommender_SLIMElasticNet.RECOMMENDER_NAME + ".zip" )

In [10]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython


In [11]:
import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x7f6686c61390>)

In [12]:
#prepare the environment to run Cython code
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1822,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:620:
/opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization_Cyth

In [13]:
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython_Epoch import MatrixFactorization_Cython_Epoch

In [ ]:
#from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython_Epoch import MatrixFactorization_Cython_Epoch

recommender_SLIM_BPR_Cython = SLIM_BPR_Cython(URM_train)
recommender_SLIM_BPR_Cython.fit(epochs=650, sgd_mode = "sgd", topK = 483, lambda_i = 0.0006712905081189398, 
                lambda_j = 0.06584150350451998, learning_rate = 0.0036482363905043207)
#evaluator_valid.evaluateRecommender(recommender)

In [ ]:
recommender_SLIM_BPR_Cython.save_model(output_folder_path, file_name = recommender_SLIM_BPR_Cython.RECOMMENDER_NAME + ".zip" )

In [ ]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
#ItemKNNCustomSimilarityRecommender is a recommender that supports a costum similary 

for alpha in np.arange(0.0, 1.1 , 0.1):
    print("alpha = ",alpha)
    new_similarity = (1 - alpha) * recommender_SLIMElasticNet.W_sparse + alpha * recommender_SLIM_BPR_Cython.W_sparse

    recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
    recommender_object.fit(new_similarity)

    result_df, _ = evaluator_valid.evaluateRecommender(recommender_object)
    print("MAP = ",result_df["MAP"])
    print("---------")

In [ ]:
for alpha in np.arange(0.0, 0.2 , 0.01):
    print("alpha = ",alpha)
    new_similarity = (1 - alpha) * recommender_SLIMElasticNet.W_sparse + alpha * recommender_SLIM_BPR_Cython.W_sparse

    recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
    recommender_object.fit(new_similarity)

    result_df, _ = evaluator_valid.evaluateRecommender(recommender_object)
    print("MAP = ",result_df["MAP"])
    print("---------")

In [ ]:
#best model - alpha = 0.03
alpha = 0.03
new_similarity = (1 - alpha) * recommender_SLIMElasticNet.W_sparse + alpha * recommender_SLIM_BPR_Cython.W_sparse

recommender = ItemKNNCustomSimilarityRecommender(URM_all)
recommender.fit(new_similarity)

TRAIN THE SINGLE MODELS WITH THE URM_all AND THEN MERGE THE SIMILARITY MATRICES

In [15]:
recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_all)
recommender_SLIMElasticNet.fit(epochs = 500, l1_ratio = 0.0023170159712850467, alpha = 0.09078974149197175, 
                positive_only = True, topK = 363)

SLIMElasticNetRecommender: Processed 1160 ( 6.4%) in 5.00 min. Items per second: 3.86
SLIMElasticNetRecommender: Processed 2344 (13.0%) in 10.01 min. Items per second: 3.90
SLIMElasticNetRecommender: Processed 3548 (19.6%) in 15.01 min. Items per second: 3.94
SLIMElasticNetRecommender: Processed 4750 (26.3%) in 20.01 min. Items per second: 3.95
SLIMElasticNetRecommender: Processed 5948 (32.9%) in 25.01 min. Items per second: 3.96
SLIMElasticNetRecommender: Processed 7146 (39.6%) in 30.02 min. Items per second: 3.97
SLIMElasticNetRecommender: Processed 8346 (46.2%) in 35.02 min. Items per second: 3.97
SLIMElasticNetRecommender: Processed 9549 (52.9%) in 40.02 min. Items per second: 3.98
SLIMElasticNetRecommender: Processed 10751 (59.5%) in 45.02 min. Items per second: 3.98
SLIMElasticNetRecommender: Processed 11956 (66.2%) in 50.02 min. Items per second: 3.98
SLIMElasticNetRecommender: Processed 13166 (72.9%) in 55.03 min. Items per second: 3.99
SLIMElasticNetRecommender: Processed 1436

In [14]:
recommender_SLIM_BPR_Cython = SLIM_BPR_Cython(URM_all)
recommender_SLIM_BPR_Cython.fit(epochs=650, sgd_mode = "sgd", topK = 483, lambda_i = 0.0006712905081189398, 
                lambda_j = 0.06584150350451998, learning_rate = 0.0036482363905043207)

SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 16718.00 MB (92.87%) of 18001.00 MB, required is 1304.51 MB. Using dense matrix.
Processed 13650 (100.0%) in 1.51 sec. BPR loss is 5.31E-01. Sample per second: 9037
SLIM_BPR_Recommender: Epoch 1 of 650. Elapsed time 0.70 sec
Processed 13650 (100.0%) in 1.21 sec. BPR loss is 1.87E+00. Sample per second: 11273
SLIM_BPR_Recommender: Epoch 2 of 650. Elapsed time 1.40 sec
Processed 13650 (100.0%) in 0.91 sec. BPR loss is 3.11E+00. Sample per second: 15011
SLIM_BPR_Recommender: Epoch 3 of 650. Elapsed time 2.10 sec
Processed 13650 (100.0%) in 1.60 sec. BPR loss is 4.17E+00. Sample per second: 8522
SLIM_BPR_Recommender: Epoch 4 of 650. Elapsed time 2.79 sec
Processed 13650 (100.0%) in 1.32 sec. BPR loss is 5.20E+00. Sample per second: 10373
SLIM_BPR_Recommender: Epoch 5 of 650. Elapsed time 3.51 sec
Processed 13650 (100.0%) in 1.03 sec. BPR loss is 6.13E+00. Sample per second: 13291
SLIM_BPR_Recommender: Epoch 6

In [17]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

#best model - alpha = 0.03
alpha = 0.03
new_similarity = (1 - alpha) * recommender_SLIMElasticNet.W_sparse + alpha * recommender_SLIM_BPR_Cython.W_sparse

recommender = ItemKNNCustomSimilarityRecommender(URM_all)
recommender.fit(new_similarity)

In [18]:
test_users = pd.read_csv('../input/recommender-system-2021-challenge-polimi/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
13645,13645
13646,13646
13647,13647
13648,13648


In [19]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user,cutoff = 10))

In [20]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('submission.csv', index=False)